In [3]:
import pandas as pd
import plotly.express as px
import dash
import dash_bootstrap_components as dbc
from dash import dcc, html
from dash.dependencies import Input, Output

# Load dataset
df = pd.read_csv("FreshMart Analytics - Orders.csv")

# Compute additional metrics
df['Profit_Margin'] = df['Profit'] / df['Total_Sales'] * 100

# Initialize Dash app
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

# Define Layout
app.layout = dbc.Container([
    dbc.Row([
        # Left Column (Title, KPIs, Dropdown)
        dbc.Col([
            html.H1("FreshMart Sales Dashboard", className="text-center mb-4"),
            
            dbc.Card([
                dbc.CardBody([
                    html.H4("Total Sales", className="card-title"),
                    html.H2(f"${df['Total_Sales'].sum():,.2f}", className="card-text")
                ])
            ], className="mb-3"),
            
            dbc.Card([
                dbc.CardBody([
                    html.H4("Total Profit", className="card-title"),
                    html.H2(f"${df['Profit'].sum():,.2f}", className="card-text")
                ])
            ], className="mb-3"),
            
            dcc.Dropdown(
                id='region_filter',
                options=[{'label': region, 'value': region} for region in df['Region_ID'].unique()],
                value=df['Region_ID'].unique()[0],
                clearable=False,
                className="mb-3"
            )
        ], width=4),
        
        # Right Column (Graphs)
        dbc.Col([
            dcc.Graph(id='sales_trend'),
            dcc.Graph(id='top_products'),
            dcc.Graph(id='category_sales'),
            dcc.Graph(id='hourly_sales_heatmap'),
            dcc.Graph(id='profit_margin_trend'),
            dcc.Graph(id='sales_vs_returns')
        ], width=8)
    ])
], fluid=True)

# Callbacks to update graphs
@app.callback(
    Output('sales_trend', 'figure'),
    [Input('region_filter', 'value')]
)
def update_sales_trend(selected_region):
    filtered_df = df[df['Region_ID'] == selected_region]
    fig = px.line(filtered_df, x='Month', y='Total_Sales', color='Year', title=f'Sales Trend for Region {selected_region}')
    return fig

@app.callback(
    Output('top_products', 'figure'),
    [Input('region_filter', 'value')]
)
def update_top_products(selected_region):
    filtered_df = df[df['Region_ID'] == selected_region]
    top_products = filtered_df.groupby('Product_ID')['Total_Sales'].sum().nlargest(5).reset_index()
    fig = px.bar(top_products, x='Product_ID', y='Total_Sales', title=f'Top 5 Products in Region {selected_region}')
    return fig

@app.callback(
    Output('category_sales', 'figure'),
    [Input('region_filter', 'value')]
)
def update_category_sales(selected_region):
    filtered_df = df[df['Region_ID'] == selected_region]
    category_sales = filtered_df.groupby('Product_ID')['Total_Sales'].sum().reset_index()
    fig = px.pie(category_sales, names='Product_ID', values='Total_Sales', title='Sales by Product Category')
    return fig

@app.callback(
    Output('hourly_sales_heatmap', 'figure'),
    [Input('region_filter', 'value')]
)
def update_hourly_sales_heatmap(selected_region):
    filtered_df = df[df['Region_ID'] == selected_region]
    fig = px.histogram(filtered_df, x='Hour', y='Total_Sales', title='Hourly Sales Distribution', nbins=24)
    return fig

@app.callback(
    Output('profit_margin_trend', 'figure'),
    [Input('region_filter', 'value')]
)
def update_profit_margin_trend(selected_region):
    filtered_df = df[df['Region_ID'] == selected_region]
    fig = px.line(filtered_df, x='Month', y='Profit_Margin', color='Year', title='Profit Margin Trend')
    return fig

@app.callback(
    Output('sales_vs_returns', 'figure'),
    [Input('region_filter', 'value')]
)
def update_sales_vs_returns(selected_region):
    filtered_df = df[df['Region_ID'] == selected_region]
    return_data = filtered_df.groupby('Return_Flag')['Total_Sales'].sum().reset_index()
    fig = px.bar(return_data, x='Return_Flag', y='Total_Sales', title='Sales vs Returns')
    return fig

if __name__ == '__main__':
    app.run_server(debug=True)


In [10]:
import pandas as pd
import plotly.express as px
import dash
import dash_bootstrap_components as dbc
from dash import dcc, html
from dash.dependencies import Input, Output

# Load dataset
df = pd.read_csv("FreshMart Analytics - Orders.csv")

# Compute additional metrics
df['Profit_Margin'] = df['Profit'] / df['Total_Sales'] * 100

df['Order_Count'] = df['Order_ID'].nunique()  # Count unique orders
df['Average_Basket_Size'] = df['Quantity_Sold'] / df['Order_Count']
df['Sales_Growth'] = df['Total_Sales'].pct_change() * 100

# Categorize sales periods
def categorize_time(hour):
    if 6 <= hour < 12:
        return "Morning"
    elif 12 <= hour < 18:
        return "Afternoon"
    elif 18 <= hour < 24:
        return "Evening"
    else:
        return "Night"

df['Sales_Period'] = df['Hour'].apply(categorize_time)

# Initialize Dash app
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

# Define Layout
app.layout = dbc.Container([
    dbc.Row([
        # Left Column (Title, KPIs, Dropdown)
        dbc.Col([
            html.H1("FreshMart Sales Dashboard", className="text-center mb-4"),
            
            dbc.Card([
                dbc.CardBody([
                    html.H4("Total Sales", className="card-title"),
                    html.H2(f"${df['Total_Sales'].sum():,.2f}", className="card-text")
                ])
            ], className="mb-3"),
            
            dbc.Card([
                dbc.CardBody([
                    html.H4("Total Profit", className="card-title"),
                    html.H2(f"${df['Profit'].sum():,.2f}", className="card-text")
                ])
            ], className="mb-3"),
            
            dbc.Card([
                dbc.CardBody([
                    html.H4("Average Basket Size", className="card-title"),
                    html.H2(f\"{df['Sales_Growth'].mean():.2f}%\", className=\"card-text\")
                ])
            ], className="mb-3"),
            
            dbc.Card([
                dbc.CardBody([
                    html.H4("Sales Growth Rate", className="card-title"),
                    html.H2(f"{df['Sales_Growth'].mean():.2f}%", className="card-text")
                ])
            ], className="mb-3"),
            
            dcc.Dropdown(
                id='region_filter',
                options=[{'label': region, 'value': region} for region in df['Region_ID'].unique()],
                value=df['Region_ID'].unique()[0],
                clearable=False,
                className="mb-3"
            )
        ], width=4),
        
        # Vertical Line Separator
        dbc.Col(html.Div(style={'border-left': '2px solid #ddd', 'height': '100%'}), width=1),
        
        # Right Column (Graphs)
        dbc.Col([
            dcc.Graph(id='sales_trend'),
            dcc.Graph(id='top_products'),
            dcc.Graph(id='category_sales'),
            dcc.Graph(id='hourly_sales_heatmap'),
            dcc.Graph(id='profit_margin_trend'),
            dcc.Graph(id='sales_vs_returns')
        ], width=7)
    ])
], fluid=True)

# Callbacks to update graphs
@app.callback(
    Output('sales_trend', 'figure'),
    [Input('region_filter', 'value')]
)
def update_sales_trend(selected_region):
    filtered_df = df[df['Region_ID'] == selected_region]
    fig = px.line(filtered_df, x='Month', y='Total_Sales', color='Year', title=f'Sales Trend for Region {selected_region}')
    return fig

@app.callback(
    Output('top_products', 'figure'),
    [Input('region_filter', 'value')]
)
def update_top_products(selected_region):
    filtered_df = df[df['Region_ID'] == selected_region]
    top_products = filtered_df.groupby('Product_ID')['Total_Sales'].sum().nlargest(5).reset_index()
    fig = px.bar(top_products, x='Product_ID', y='Total_Sales', title=f'Top 5 Products in Region {selected_region}')
    return fig

@app.callback(
    Output('category_sales', 'figure'),
    [Input('region_filter', 'value')]
)
def update_category_sales(selected_region):
    filtered_df = df[df['Region_ID'] == selected_region]
    category_sales = filtered_df.groupby('Product_ID')['Total_Sales'].sum().reset_index()
    fig = px.pie(category_sales, names='Product_ID', values='Total_Sales', title='Sales by Product Category')
    return fig

@app.callback(
    Output('hourly_sales_heatmap', 'figure'),
    [Input('region_filter', 'value')]
)
def update_hourly_sales_heatmap(selected_region):
    filtered_df = df[df['Region_ID'] == selected_region]
    fig = px.histogram(filtered_df, x='Hour', y='Total_Sales', title='Hourly Sales Distribution', nbins=24)
    return fig

@app.callback(
    Output('profit_margin_trend', 'figure'),
    [Input('region_filter', 'value')]
)
def update_profit_margin_trend(selected_region):
    filtered_df = df[df['Region_ID'] == selected_region]
    fig = px.line(filtered_df, x='Month', y='Profit_Margin', color='Year', title='Profit Margin Trend')
    return fig

@app.callback(
    Output('sales_vs_returns', 'figure'),
    [Input('region_filter', 'value')]
)
def update_sales_vs_returns(selected_region):
    filtered_df = df[df['Region_ID'] == selected_region]
    return_data = filtered_df.groupby('Return_Flag')['Total_Sales'].sum().reset_index()
    fig = px.bar(return_data, x='Return_Flag', y='Total_Sales', title='Sales vs Returns')
    return fig

if __name__ == '__main__':
    app.run_server(debug=True)


SyntaxError: unterminated string literal (detected at line 65) (1160429057.py, line 65)

In [11]:
import pandas as pd
import plotly.express as px
import dash
import dash_bootstrap_components as dbc
from dash import dcc, html
from dash.dependencies import Input, Output

# Load dataset
df = pd.read_csv("FreshMart Analytics - Orders.csv")

# Compute additional metrics
df['Profit_Margin'] = df['Profit'] / df['Total_Sales'] * 100

df['Order_Count'] = df['Order_ID'].nunique()  # Count unique orders
df['Average_Basket_Size'] = df['Quantity_Sold'] / df['Order_Count']
df['Sales_Growth'] = df['Total_Sales'].pct_change() * 100

# Categorize sales periods
def categorize_time(hour):
    if 6 <= hour < 12:
        return "Morning"
    elif 12 <= hour < 18:
        return "Afternoon"
    elif 18 <= hour < 24:
        return "Evening"
    else:
        return "Night"

df['Sales_Period'] = df['Hour'].apply(categorize_time)

# Initialize Dash app
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

# Define Layout
app.layout = dbc.Container([
    dbc.Row([
        # Left Column (Title, KPIs, Dropdown)
        dbc.Col([
            html.H1("FreshMart Sales Dashboard", className="text-center mb-4"),
            
            dbc.Card([
                dbc.CardBody([
                    html.H4("Total Sales", className="card-title"),
                    html.H2(f"${df['Total_Sales'].sum():,.2f}", className="card-text")
                ])
            ], className="mb-3"),
            
            dbc.Card([
                dbc.CardBody([
                    html.H4("Total Profit", className="card-title"),
                    html.H2(f"${df['Profit'].sum():,.2f}", className="card-text")
                ])
            ], className="mb-3"),
            
            dbc.Card([
                dbc.CardBody([
                    html.H4("Average Basket Size", className="card-title"),
                    html.H2(f"{df['Average_Basket_Size'].mean():.2f}", className="card-text")
                ])
            ], className="mb-3"),
            
            dbc.Card([
                dbc.CardBody([
                    html.H4("Sales Growth Rate", className="card-title"),
                    html.H2(f"{df['Sales_Growth'].mean():.2f}%", className="card-text")
                ])
            ], className="mb-3"),
            
            dcc.Dropdown(
                id='region_filter',
                options=[{'label': region, 'value': region} for region in df['Region_ID'].unique()],
                value=df['Region_ID'].unique()[0],
                clearable=False,
                className="mb-3"
            )
        ], width=4),
        
        # Vertical Line Separator
        dbc.Col(html.Div(style={'border-left': '2px solid #ddd', 'height': '100%'}), width=1),
        
        # Right Column (Graphs)
        dbc.Col([
            dcc.Graph(id='sales_trend'),
            dcc.Graph(id='top_products'),
            dcc.Graph(id='category_sales'),
            dcc.Graph(id='hourly_sales_heatmap'),
            dcc.Graph(id='profit_margin_trend'),
            dcc.Graph(id='sales_vs_returns')
        ], width=7)
    ])
], fluid=True)

# Callbacks to update graphs
@app.callback(
    Output('sales_trend', 'figure'),
    [Input('region_filter', 'value')]
)
def update_sales_trend(selected_region):
    filtered_df = df[df['Region_ID'] == selected_region]
    fig = px.line(filtered_df, x='Month', y='Total_Sales', color='Year', title=f'Sales Trend for Region {selected_region}')
    return fig

@app.callback(
    Output('top_products', 'figure'),
    [Input('region_filter', 'value')]
)
def update_top_products(selected_region):
    filtered_df = df[df['Region_ID'] == selected_region]
    top_products = filtered_df.groupby('Product_ID')['Total_Sales'].sum().nlargest(5).reset_index()
    fig = px.bar(top_products, x='Product_ID', y='Total_Sales', title=f'Top 5 Products in Region {selected_region}')
    return fig

@app.callback(
    Output('category_sales', 'figure'),
    [Input('region_filter', 'value')]
)
def update_category_sales(selected_region):
    filtered_df = df[df['Region_ID'] == selected_region]
    category_sales = filtered_df.groupby('Product_ID')['Total_Sales'].sum().reset_index()
    fig = px.pie(category_sales, names='Product_ID', values='Total_Sales', title='Sales by Product Category')
    return fig

@app.callback(
    Output('hourly_sales_heatmap', 'figure'),
    [Input('region_filter', 'value')]
)
def update_hourly_sales_heatmap(selected_region):
    filtered_df = df[df['Region_ID'] == selected_region]
    fig = px.histogram(filtered_df, x='Hour', y='Total_Sales', title='Hourly Sales Distribution', nbins=24)
    return fig

@app.callback(
    Output('profit_margin_trend', 'figure'),
    [Input('region_filter', 'value')]
)
def update_profit_margin_trend(selected_region):
    filtered_df = df[df['Region_ID'] == selected_region]
    fig = px.line(filtered_df, x='Month', y='Profit_Margin', color='Year', title='Profit Margin Trend')
    return fig

@app.callback(
    Output('sales_vs_returns', 'figure'),
    [Input('region_filter', 'value')]
)
def update_sales_vs_returns(selected_region):
    filtered_df = df[df['Region_ID'] == selected_region]
    return_data = filtered_df.groupby('Return_Flag')['Total_Sales'].sum().reset_index()
    fig = px.bar(return_data, x='Return_Flag', y='Total_Sales', title='Sales vs Returns')
    return fig

if __name__ == '__main__':
    app.run_server(debug=True)


In [6]:
import pandas as pd
import plotly.express as px
import dash
import dash_bootstrap_components as dbc
from dash import dcc, html
from dash.dependencies import Input, Output

# Load dataset
df = pd.read_csv("FreshMart Analytics - Orders.csv")

# Compute additional metrics
df['Profit_Margin'] = df['Profit'] / df['Total_Sales'] * 100

df['Order_Count'] = df['Order_ID'].nunique()  # Count unique orders
df['Average_Basket_Size'] = df['Quantity_Sold'] / df['Order_Count']
df['Sales_Growth'] = df['Total_Sales'].pct_change() * 100

# Categorize sales periods
def categorize_time(hour):
    if 6 <= hour < 12:
        return "Morning"
    elif 12 <= hour < 18:
        return "Afternoon"
    elif 18 <= hour < 24:
        return "Evening"
    else:
        return "Night"

df['Sales_Period'] = df['Hour'].apply(categorize_time)

# Initialize Dash app
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

# Define Layout
app.layout = dbc.Container([
    dbc.Row([
        # Left Column (Title, KPIs, Dropdown)
        dbc.Col([
            html.H1("FreshMart Sales Dashboard", className="text-center mb-4"),
            
            dbc.Card([
                dbc.CardBody([
                    html.H4("Total Sales", className="card-title"),
                    html.H2(f"${df['Total_Sales'].sum():,.2f}", className="card-text")
                ])
            ], className="mb-3"),
            
            dbc.Card([
                dbc.CardBody([
                    html.H4("Total Profit", className="card-title"),
                    html.H2(f"${df['Profit'].sum():,.2f}", className="card-text")
                ])
            ], className="mb-3"),
            
            dbc.Card([
                dbc.CardBody([
                    html.H4("Average Basket Size", className="card-title"),
                    html.H2(f"{df['Average_Basket_Size'].mean():.2f}", className="card-text")
                ])
            ], className="mb-3"),
            
            dbc.Card([
                dbc.CardBody([
                    html.H4("Sales Growth Rate", className="card-title"),
                    html.H2(f"{df['Sales_Growth'].mean():.2f}%", className="card-text")
                ])
            ], className="mb-3"),
            
            # Title for Dropdown and Dropdown
            html.Div([
                html.H5("Select Region", className="mb-1"),
                dcc.Dropdown(
                    id='region_filter',
                    options=[{'label': region, 'value': region} for region in df['Region_ID'].unique()],
                    value=df['Region_ID'].unique()[0],
                    clearable=False
                )
            ], className="mb-3")
        ], width=4),
        
        # Vertical Line Separator
        dbc.Col(html.Div(style={'border-left': '2px solid #ddd', 'height': '100%'}), width=1),
        
        # Right Column (Graphs)
        dbc.Col([
            dcc.Graph(id='sales_trend'),
            dcc.Graph(id='top_products'),
            dcc.Graph(id='category_sales'),
            dcc.Graph(id='hourly_sales_heatmap'),
            dcc.Graph(id='profit_margin_trend'),
            dcc.Graph(id='sales_vs_returns')
        ], width=7)
    ])
], fluid=True)

# Callbacks to update graphs
@app.callback(
    Output('sales_trend', 'figure'),
    [Input('region_filter', 'value')]
)
def update_sales_trend(selected_region):
    filtered_df = df[df['Region_ID'] == selected_region]
    fig = px.line(filtered_df, x='Month', y='Total_Sales', color='Year', title=f'Sales Trend for Region {selected_region}')
    return fig

@app.callback(
    Output('top_products', 'figure'),
    [Input('region_filter', 'value')]
)
def update_top_products(selected_region):
    filtered_df = df[df['Region_ID'] == selected_region]
    top_products = filtered_df.groupby('Product_ID')['Total_Sales'].sum().nlargest(5).reset_index()
    fig = px.bar(top_products, x='Product_ID', y='Total_Sales', title=f'Top 5 Products in Region {selected_region}')
    return fig

@app.callback(
    Output('category_sales', 'figure'),
    [Input('region_filter', 'value')]
)
def update_category_sales(selected_region):
    filtered_df = df[df['Region_ID'] == selected_region]
    category_sales = filtered_df.groupby('Product_ID')['Total_Sales'].sum().reset_index()
    fig = px.pie(category_sales, names='Product_ID', values='Total_Sales', title='Sales by Product Category')
    return fig

@app.callback(
    Output('hourly_sales_heatmap', 'figure'),
    [Input('region_filter', 'value')]
)
def update_hourly_sales_heatmap(selected_region):
    filtered_df = df[df['Region_ID'] == selected_region]
    fig = px.histogram(filtered_df, x='Hour', y='Total_Sales', title='Hourly Sales Distribution', nbins=24)
    return fig

@app.callback(
    Output('profit_margin_trend', 'figure'),
    [Input('region_filter', 'value')]
)
def update_profit_margin_trend(selected_region):
    filtered_df = df[df['Region_ID'] == selected_region]
    fig = px.line(filtered_df, x='Month', y='Profit_Margin', color='Year', title='Profit Margin Trend')
    return fig

@app.callback(
    Output('sales_vs_returns', 'figure'),
    [Input('region_filter', 'value')]
)
def update_sales_vs_returns(selected_region):
    filtered_df = df[df['Region_ID'] == selected_region]
    return_data = filtered_df.groupby('Return_Flag')['Total_Sales'].sum().reset_index()
    fig = px.bar(return_data, x='Return_Flag', y='Total_Sales', title='Sales vs Returns')
    return fig

if __name__ == '__main__':
    app.run_server(debug=True)


In [7]:
import pandas as pd
import plotly.express as px
import dash
import dash_bootstrap_components as dbc
from dash import dcc, html
from dash.dependencies import Input, Output

# Load dataset
df = pd.read_csv("FreshMart Analytics - Orders.csv")

# Compute additional metrics
df['Profit_Margin'] = df['Profit'] / df['Total_Sales'] * 100

# Calculate Order_Count as number of unique orders.
order_count = df['Order_ID'].nunique()  
df['Average_Basket_Size'] = df['Quantity_Sold'] / order_count
df['Sales_Growth'] = df['Total_Sales'].pct_change() * 100

# Categorize sales periods
def categorize_time(hour):
    if 6 <= hour < 12:
        return "Morning"
    elif 12 <= hour < 18:
        return "Afternoon"
    elif 18 <= hour < 24:
        return "Evening"
    else:
        return "Night"

df['Sales_Period'] = df['Hour'].apply(categorize_time)

# Initialize Dash app
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

# Define Layout
app.layout = dbc.Container([
    dbc.Row([
        # Left Column (Title, KPIs, Dropdown)
        dbc.Col([
            html.H1("FreshMart Sales Dashboard", className="text-center mb-4"),
            
            dbc.Card([
                dbc.CardBody([
                    html.H4("Total Sales", className="card-title"),
                    html.H2(f"${df['Total_Sales'].sum():,.2f}", className="card-text")
                ])
            ], className="mb-3"),
            
            dbc.Card([
                dbc.CardBody([
                    html.H4("Total Profit", className="card-title"),
                    html.H2(f"${df['Profit'].sum():,.2f}", className="card-text")
                ])
            ], className="mb-3"),
            
            dbc.Card([
                dbc.CardBody([
                    html.H4("Average Basket Size", className="card-title"),
                    html.H2(f"{df['Average_Basket_Size'].mean():.2f}", className="card-text")
                ])
            ], className="mb-3"),
            
            dbc.Card([
                dbc.CardBody([
                    html.H4("Sales Growth Rate", className="card-title"),
                    html.H2(f"{df['Sales_Growth'].mean():.2f}%", className="card-text")
                ])
            ], className="mb-3"),
            
            # Title for Dropdown and Dropdown
            html.Div([
                html.H5("Select Region", className="mb-1"),
                dcc.Dropdown(
                    id='region_filter',
                    options=[{'label': region, 'value': region} for region in df['Region_ID'].unique()],
                    value=df['Region_ID'].unique()[0],
                    clearable=False
                )
            ], className="mb-3")
        ], width=4),
        
        # Vertical Line Separator
        dbc.Col(html.Div(style={'border-left': '2px solid #ddd', 'height': '100%'}), width=1),
        
        # Right Column (Graphs)
        dbc.Col([
            dcc.Graph(id='sales_trend'),
            dcc.Graph(id='top_products'),
            dcc.Graph(id='category_sales'),
            # Heat Map graph: Total Sales by Day of Week and Hour
            dcc.Graph(id='sales_heatmap'),
            dcc.Graph(id='profit_margin_trend'),
            dcc.Graph(id='sales_vs_returns'),
            # Scatter Plot: Profit vs Total Sales
            dcc.Graph(id='sales_profit_scatter')
        ], width=7)
    ])
], fluid=True)

# Callbacks to update graphs
@app.callback(
    Output('sales_trend', 'figure'),
    [Input('region_filter', 'value')]
)
def update_sales_trend(selected_region):
    filtered_df = df[df['Region_ID'] == selected_region]
    fig = px.line(filtered_df, x='Month', y='Total_Sales', color='Year', title=f'Sales Trend for Region {selected_region}')
    return fig

@app.callback(
    Output('top_products', 'figure'),
    [Input('region_filter', 'value')]
)
def update_top_products(selected_region):
    filtered_df = df[df['Region_ID'] == selected_region]
    top_products = filtered_df.groupby('Product_ID')['Total_Sales'].sum().nlargest(5).reset_index()
    fig = px.bar(top_products, x='Product_ID', y='Total_Sales', title=f'Top 5 Products in Region {selected_region}')
    return fig

@app.callback(
    Output('category_sales', 'figure'),
    [Input('region_filter', 'value')]
)
def update_category_sales(selected_region):
    filtered_df = df[df['Region_ID'] == selected_region]
    category_sales = filtered_df.groupby('Product_ID')['Total_Sales'].sum().reset_index()
    fig = px.pie(category_sales, names='Product_ID', values='Total_Sales', title='Sales by Product Category')
    return fig

@app.callback(
    Output('sales_heatmap', 'figure'),
    [Input('region_filter', 'value')]
)
def update_sales_heatmap(selected_region):
    filtered_df = df[df['Region_ID'] == selected_region]
    # Create a pivot table: rows = Day_of_Week, columns = Hour, values = Total_Sales
    pivot = filtered_df.pivot_table(index='Day_of_Week', columns='Hour', values='Total_Sales', aggfunc='sum', fill_value=0)
    # Order days of week
    day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
    pivot = pivot.reindex(day_order).dropna(how='all')
    fig = px.imshow(pivot, text_auto=True, aspect='auto', title='Heatmap of Total Sales by Day and Hour')
    fig.update_xaxes(title_text='Hour')
    fig.update_yaxes(title_text='Day of Week')
    return fig

@app.callback(
    Output('profit_margin_trend', 'figure'),
    [Input('region_filter', 'value')]
)
def update_profit_margin_trend(selected_region):
    filtered_df = df[df['Region_ID'] == selected_region]
    fig = px.line(filtered_df, x='Month', y='Profit_Margin', color='Year', title='Profit Margin Trend')
    return fig

@app.callback(
    Output('sales_vs_returns', 'figure'),
    [Input('region_filter', 'value')]
)
def update_sales_vs_returns(selected_region):
    filtered_df = df[df['Region_ID'] == selected_region]
    return_data = filtered_df.groupby('Return_Flag')['Total_Sales'].sum().reset_index()
    fig = px.bar(return_data, x='Return_Flag', y='Total_Sales', title='Sales vs Returns')
    return fig

@app.callback(
    Output('sales_profit_scatter', 'figure'),
    [Input('region_filter', 'value')]
)
def update_sales_profit_scatter(selected_region):
    filtered_df = df[df['Region_ID'] == selected_region]
    fig = px.scatter(filtered_df, x='Total_Sales', y='Profit', color='Year', size='Quantity_Sold',
                     hover_data=['Product_ID'], title='Scatter Plot of Profit vs Total Sales')
    return fig

if __name__ == '__main__':
    app.run_server(debug=True)


In [8]:
import pandas as pd
import plotly.express as px
import dash
import dash_bootstrap_components as dbc
from dash import dcc, html
from dash.dependencies import Input, Output

# Load dataset
df = pd.read_csv("FreshMart Analytics - Orders.csv")

# Compute additional metrics
df['Profit_Margin'] = df['Profit'] / df['Total_Sales'] * 100

# Calculate Order_Count as number of unique orders.
order_count = df['Order_ID'].nunique()  
df['Average_Basket_Size'] = df['Quantity_Sold'] / order_count
df['Sales_Growth'] = df['Total_Sales'].pct_change() * 100

# Categorize sales periods
def categorize_time(hour):
    if 6 <= hour < 12:
        return "Morning"
    elif 12 <= hour < 18:
        return "Afternoon"
    elif 18 <= hour < 24:
        return "Evening"
    else:
        return "Night"

df['Sales_Period'] = df['Hour'].apply(categorize_time)

# Initialize Dash app
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

# Define Layout
app.layout = dbc.Container([
    dbc.Row([
        # Left Column (Title, KPIs, Dropdown)
        dbc.Col([
            html.H1("FreshMart Sales Dashboard", className="text-center mb-4"),
            
            dbc.Card([
                dbc.CardBody([
                    html.H4("Total Sales", className="card-title"),
                    html.H2(f"${df['Total_Sales'].sum():,.2f}", className="card-text")
                ])
            ], className="mb-3"),
            
            dbc.Card([
                dbc.CardBody([
                    html.H4("Total Profit", className="card-title"),
                    html.H2(f"${df['Profit'].sum():,.2f}", className="card-text")
                ])
            ], className="mb-3"),
            
            dbc.Card([
                dbc.CardBody([
                    html.H4("Average Basket Size", className="card-title"),
                    html.H2(f"{df['Average_Basket_Size'].mean():.2f}", className="card-text")
                ])
            ], className="mb-3"),
            
            dbc.Card([
                dbc.CardBody([
                    html.H4("Sales Growth Rate", className="card-title"),
                    html.H2(f"{df['Sales_Growth'].mean():.2f}%", className="card-text")
                ])
            ], className="mb-3"),
            
            # Title for Dropdown and Dropdown
            html.Div([
                html.H5("Select Region", className="mb-1"),
                dcc.Dropdown(
                    id='region_filter',
                    options=[{'label': region, 'value': region} for region in df['Region_ID'].unique()],
                    value=df['Region_ID'].unique()[0],
                    clearable=False
                )
            ], className="mb-3")
        ], width=4),
        
        # Vertical Line Separator
        dbc.Col(html.Div(style={'border-left': '2px solid #ddd', 'height': '100%'}), width=1),
        
        # Right Column (Graphs)
        dbc.Col([
            dcc.Graph(id='sales_trend'),
            dcc.Graph(id='top_products'),
            dcc.Graph(id='category_sales'),
            dcc.Graph(id='sales_heatmap'),
            dcc.Graph(id='profit_margin_trend'),
            dcc.Graph(id='sales_vs_returns'),
            dcc.Graph(id='sales_profit_scatter')
        ], width=7)
    ])
], fluid=True)

# Callbacks to update graphs
@app.callback(
    Output('sales_trend', 'figure'),
    [Input('region_filter', 'value')]
)
def update_sales_trend(selected_region):
    filtered_df = df[df['Region_ID'] == selected_region]
    # تجميع البيانات شهريًا لكل سنة وحساب مجموع المبيعات
    grouped_df = filtered_df.groupby(['Year', 'Month'])['Total_Sales'].sum().reset_index()

    fig = px.line(
        grouped_df, 
        x='Month', 
        y='Total_Sales', 
        color='Year', 
        markers=True,
        title=f'Sales Trend for Region {selected_region}'
    )
    fig.update_xaxes(dtick=1)  # لضبط المحور السيني بحيث يظهر كل شهر
    return fig

@app.callback(
    Output('top_products', 'figure'),
    [Input('region_filter', 'value')]
)
def update_top_products(selected_region):
    filtered_df = df[df['Region_ID'] == selected_region]
    top_products = filtered_df.groupby('Product_ID')['Total_Sales'].sum().nlargest(5).reset_index()
    fig = px.bar(top_products, x='Product_ID', y='Total_Sales', title=f'Top 5 Products in Region {selected_region}')
    return fig

@app.callback(
    Output('category_sales', 'figure'),
    [Input('region_filter', 'value')]
)
def update_category_sales(selected_region):
    filtered_df = df[df['Region_ID'] == selected_region]
    category_sales = filtered_df.groupby('Product_ID')['Total_Sales'].sum().reset_index()
    fig = px.pie(category_sales, names='Product_ID', values='Total_Sales', title='Sales by Product Category')
    return fig

@app.callback(
    Output('sales_heatmap', 'figure'),
    [Input('region_filter', 'value')]
)
def update_sales_heatmap(selected_region):
    filtered_df = df[df['Region_ID'] == selected_region]
    # Create a pivot table: rows = Day_of_Week, columns = Hour, values = Total_Sales
    pivot = filtered_df.pivot_table(index='Day_of_Week', columns='Hour', values='Total_Sales', aggfunc='sum', fill_value=0)
    # Order days of week
    day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
    pivot = pivot.reindex(day_order).dropna(how='all')
    fig = px.imshow(pivot, text_auto=True, aspect='auto', title='Heatmap of Total Sales by Day and Hour')
    fig.update_xaxes(title_text='Hour')
    fig.update_yaxes(title_text='Day of Week')
    return fig

@app.callback(
    Output('profit_margin_trend', 'figure'),
    [Input('region_filter', 'value')]
)
def update_profit_margin_trend(selected_region):
    filtered_df = df[df['Region_ID'] == selected_region]
    # تجميع البيانات شهريًا لكل سنة وحساب متوسط هامش الربح
    grouped_df = filtered_df.groupby(['Year', 'Month'])['Profit_Margin'].mean().reset_index()

    fig = px.line(
        grouped_df,
        x='Month',
        y='Profit_Margin',
        color='Year',
        markers=True,
        title='Profit Margin Trend'
    )
    fig.update_xaxes(dtick=1)
    return fig

@app.callback(
    Output('sales_vs_returns', 'figure'),
    [Input('region_filter', 'value')]
)
def update_sales_vs_returns(selected_region):
    filtered_df = df[df['Region_ID'] == selected_region]
    return_data = filtered_df.groupby('Return_Flag')['Total_Sales'].sum().reset_index()
    fig = px.bar(return_data, x='Return_Flag', y='Total_Sales', title='Sales vs Returns')
    return fig

@app.callback(
    Output('sales_profit_scatter', 'figure'),
    [Input('region_filter', 'value')]
)
def update_sales_profit_scatter(selected_region):
    filtered_df = df[df['Region_ID'] == selected_region]
    fig = px.scatter(
        filtered_df,
        x='Total_Sales',
        y='Profit',
        color='Year',
        size='Quantity_Sold',
        hover_data=['Product_ID'],
        title='Scatter Plot of Profit vs Total Sales'
    )
    return fig

if __name__ == '__main__':
    app.run_server(debug=True)


In [9]:
import pandas as pd
import plotly.express as px
import dash
import dash_bootstrap_components as dbc
from dash import dcc, html
from dash.dependencies import Input, Output

# Load dataset
df = pd.read_csv("FreshMart Analytics - Orders.csv")

# Compute additional metrics
df['Profit_Margin'] = df['Profit'] / df['Total_Sales'] * 100

order_count = df['Order_ID'].nunique()
df['Average_Basket_Size'] = df['Quantity_Sold'] / order_count
df['Sales_Growth'] = df['Total_Sales'].pct_change() * 100

# Categorize sales periods
def categorize_time(hour):
    if 6 <= hour < 12:
        return "Morning"
    elif 12 <= hour < 18:
        return "Afternoon"
    elif 18 <= hour < 24:
        return "Evening"
    else:
        return "Night"

df['Sales_Period'] = df['Hour'].apply(categorize_time)

# Initialize Dash app
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

# Define Layout
app.layout = dbc.Container([
    dbc.Row([
        # Left Column (Title, KPIs, Dropdown)
        dbc.Col([
            html.H1("FreshMart Sales Dashboard", className="text-center mb-4"),
            
            dbc.Card([
                dbc.CardBody([
                    html.H4("Total Sales", className="card-title"),
                    html.H2(f"${df['Total_Sales'].sum():,.2f}", className="card-text")
                ])
            ], className="mb-3"),
            
            dbc.Card([
                dbc.CardBody([
                    html.H4("Total Profit", className="card-title"),
                    html.H2(f"${df['Profit'].sum():,.2f}", className="card-text")
                ])
            ], className="mb-3"),
            
            dbc.Card([
                dbc.CardBody([
                    html.H4("Average Basket Size", className="card-title"),
                    html.H2(f"{df['Average_Basket_Size'].mean():.2f}", className="card-text")
                ])
            ], className="mb-3"),
            
            dbc.Card([
                dbc.CardBody([
                    html.H4("Sales Growth Rate", className="card-title"),
                    html.H2(f\"{df['Sales_Growth'].mean():.2f}%\", className=\"card-text\")
                ])
            ], className=\"mb-3\"),

            # Title for Dropdown and Dropdown
            html.Div([
                html.H5(\"Select Region\", className=\"mb-1\"),
                dcc.Dropdown(
                    id='region_filter',
                    options=[{'label': region, 'value': region} for region in df['Region_ID'].unique()],
                    value=df['Region_ID'].unique()[0],
                    clearable=False
                )
            ], className=\"mb-3\")
        ], width=4),
        
        # Vertical Line Separator
        dbc.Col(html.Div(style={'border-left': '2px solid #ddd', 'height': '100%'}), width=1),
        
        # Right Column (Graphs)
        dbc.Col([
            dcc.Graph(id='sales_trend'),
            dcc.Graph(id='top_products'),
            dcc.Graph(id='category_sales'),
            dcc.Graph(id='sales_heatmap'),
            dcc.Graph(id='profit_margin_trend'),
            dcc.Graph(id='sales_vs_returns'),
            dcc.Graph(id='sales_profit_scatter'),
            # هنا نضيف مخطط توزيع الفترات
            dcc.Graph(id='sales_period_distribution')
        ], width=7)
    ])
], fluid=True)

# Callbacks to update graphs
@app.callback(
    Output('sales_trend', 'figure'),
    [Input('region_filter', 'value')]
)
def update_sales_trend(selected_region):
    filtered_df = df[df['Region_ID'] == selected_region]
    grouped_df = filtered_df.groupby(['Year', 'Month'])['Total_Sales'].sum().reset_index()

    fig = px.line(
        grouped_df, 
        x='Month', 
        y='Total_Sales', 
        color='Year', 
        markers=True,
        title=f'Sales Trend for Region {selected_region}'
    )
    fig.update_xaxes(dtick=1)
    return fig

@app.callback(
    Output('top_products', 'figure'),
    [Input('region_filter', 'value')]
)
def update_top_products(selected_region):
    filtered_df = df[df['Region_ID'] == selected_region]
    top_products = filtered_df.groupby('Product_ID')['Total_Sales'].sum().nlargest(5).reset_index()
    fig = px.bar(top_products, x='Product_ID', y='Total_Sales', title=f'Top 5 Products in Region {selected_region}')
    return fig

@app.callback(
    Output('category_sales', 'figure'),
    [Input('region_filter', 'value')]
)
def update_category_sales(selected_region):
    filtered_df = df[df['Region_ID'] == selected_region]
    category_sales = filtered_df.groupby('Product_ID')['Total_Sales'].sum().reset_index()
    fig = px.pie(category_sales, names='Product_ID', values='Total_Sales', title='Sales by Product Category')
    return fig

@app.callback(
    Output('sales_heatmap', 'figure'),
    [Input('region_filter', 'value')]
)
def update_sales_heatmap(selected_region):
    filtered_df = df[df['Region_ID'] == selected_region]
    pivot = filtered_df.pivot_table(index='Day_of_Week', columns='Hour', values='Total_Sales', aggfunc='sum', fill_value=0)
    day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
    pivot = pivot.reindex(day_order).dropna(how='all')
    fig = px.imshow(pivot, text_auto=True, aspect='auto', title='Heatmap of Total Sales by Day and Hour')
    fig.update_xaxes(title_text='Hour')
    fig.update_yaxes(title_text='Day of Week')
    return fig

@app.callback(
    Output('profit_margin_trend', 'figure'),
    [Input('region_filter', 'value')]
)
def update_profit_margin_trend(selected_region):
    filtered_df = df[df['Region_ID'] == selected_region]
    grouped_df = filtered_df.groupby(['Year', 'Month'])['Profit_Margin'].mean().reset_index()
    fig = px.line(
        grouped_df,
        x='Month',
        y='Profit_Margin',
        color='Year',
        markers=True,
        title='Profit Margin Trend'
    )
    fig.update_xaxes(dtick=1)
    return fig

@app.callback(
    Output('sales_vs_returns', 'figure'),
    [Input('region_filter', 'value')]
)
def update_sales_vs_returns(selected_region):
    filtered_df = df[df['Region_ID'] == selected_region]
    return_data = filtered_df.groupby('Return_Flag')['Total_Sales'].sum().reset_index()
    fig = px.bar(return_data, x='Return_Flag', y='Total_Sales', title='Sales vs Returns')
    return fig

@app.callback(
    Output('sales_profit_scatter', 'figure'),
    [Input('region_filter', 'value')]
)
def update_sales_profit_scatter(selected_region):
    filtered_df = df[df['Region_ID'] == selected_region]
    fig = px.scatter(
        filtered_df,
        x='Total_Sales',
        y='Profit',
        color='Year',
        size='Quantity_Sold',
        hover_data=['Product_ID'],
        title='Scatter Plot of Profit vs Total Sales'
    )
    return fig

@app.callback(
    Output('sales_period_distribution', 'figure'),
    [Input('region_filter', 'value')]
)
def update_sales_period_distribution(selected_region):
    filtered_df = df[df['Region_ID'] == selected_region]
    period_data = filtered_df.groupby('Sales_Period')['Total_Sales'].sum().reset_index()
    fig = px.bar(
        period_data, 
        x='Sales_Period', 
        y='Total_Sales', 
        title='Sales by Period of Day'
    )
    return fig

if __name__ == '__main__':
    app.run_server(debug=True)


SyntaxError: unexpected character after line continuation character (2089468566.py, line 65)

import pandas as pd
import plotly.express as px
import dash
import dash_bootstrap_components as dbc
from dash import dcc, html
from dash.dependencies import Input, Output

# Load dataset
df = pd.read_csv("FreshMart Analytics - Orders.csv")

# Compute additional metrics
df['Profit_Margin'] = df['Profit'] / df['Total_Sales'] * 100

df['Order_Count'] = df['Order_ID'].nunique()  # Count unique orders
df['Average_Basket_Size'] = df['Quantity_Sold'] / df['Order_Count']
df['Sales_Growth'] = df['Total_Sales'].pct_change() * 100

# Categorize sales periods
def categorize_time(hour):
    if 6 <= hour < 12:
        return "Morning"
    elif 12 <= hour < 18:
        return "Afternoon"
    elif 18 <= hour < 24:
        return "Evening"
    else:
        return "Night"

df['Sales_Period'] = df['Hour'].apply(categorize_time)

# Initialize Dash app
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

# Define Layout
app.layout = dbc.Container([
    dbc.Row([
        # Left Column (Title, KPIs, Dropdown)
        dbc.Col([
            html.H1("FreshMart Sales Dashboard", className="text-center mb-4"),
            
            dbc.Card([
                dbc.CardBody([
                    html.H4("Total Sales", className="card-title"),
                    html.H2(f"${df['Total_Sales'].sum():,.2f}", className="card-text")
                ])
            ], className="mb-3"),
            
            dbc.Card([
                dbc.CardBody([
                    html.H4("Total Profit", className="card-title"),
                    html.H2(f"${df['Profit'].sum():,.2f}", className="card-text")
                ])
            ], className="mb-3"),
            
            dbc.Card([
                dbc.CardBody([
                    html.H4("Average Basket Size", className="card-title"),
                    html.H2(f"{df['Average_Basket_Size'].mean():.2f}", className="card-text")
                ])
            ], className="mb-3"),
            
            dbc.Card([
                dbc.CardBody([
                    html.H4("Sales Growth Rate", className="card-title"),
                    html.H2(f"{df['Sales_Growth'].mean():.2f}%", className="card-text")
                ])
            ], className="mb-3"),
            
            dcc.Dropdown(
                id='region_filter',
                options=[{'label': region, 'value': region} for region in df['Region_ID'].unique()],
                value=df['Region_ID'].unique()[0],
                clearable=False,
                className="mb-3"
            )
        ], width=4),
        
        # Vertical Line Separator
        dbc.Col(html.Div(style={'border-left': '2px solid #ddd', 'height': '100%'}), width=1),
        
        # Right Column (Graphs)
        dbc.Col([
            dcc.Graph(id='sales_trend'),
            dcc.Graph(id='top_products'),
            dcc.Graph(id='category_sales'),
            dcc.Graph(id='hourly_sales_heatmap'),
            dcc.Graph(id='profit_margin_trend'),
            dcc.Graph(id='sales_vs_returns')
        ], width=7)
    ])
], fluid=True)

# Callbacks to update graphs
@app.callback(
    Output('sales_trend', 'figure'),
    [Input('region_filter', 'value')]
)
def update_sales_trend(selected_region):
    filtered_df = df[df['Region_ID'] == selected_region]
    fig = px.line(filtered_df, x='Month', y='Total_Sales', color='Year', title=f'Sales Trend for Region {selected_region}')
    return fig

@app.callback(
    Output('top_products', 'figure'),
    [Input('region_filter', 'value')]
)
def update_top_products(selected_region):
    filtered_df = df[df['Region_ID'] == selected_region]
    top_products = filtered_df.groupby('Product_ID')['Total_Sales'].sum().nlargest(5).reset_index()
    fig = px.bar(top_products, x='Product_ID', y='Total_Sales', title=f'Top 5 Products in Region {selected_region}')
    return fig

@app.callback(
    Output('category_sales', 'figure'),
    [Input('region_filter', 'value')]
)
def update_category_sales(selected_region):
    filtered_df = df[df['Region_ID'] == selected_region]
    category_sales = filtered_df.groupby('Product_ID')['Total_Sales'].sum().reset_index()
    fig = px.pie(category_sales, names='Product_ID', values='Total_Sales', title='Sales by Product Category')
    return fig

@app.callback(
    Output('hourly_sales_heatmap', 'figure'),
    [Input('region_filter', 'value')]
)
def update_hourly_sales_heatmap(selected_region):
    filtered_df = df[df['Region_ID'] == selected_region]
    fig = px.histogram(filtered_df, x='Hour', y='Total_Sales', title='Hourly Sales Distribution', nbins=24)
    return fig

@app.callback(
    Output('profit_margin_trend', 'figure'),
    [Input('region_filter', 'value')]
)
def update_profit_margin_trend(selected_region):
    filtered_df = df[df['Region_ID'] == selected_region]
    fig = px.line(filtered_df, x='Month', y='Profit_Margin', color='Year', title='Profit Margin Trend')
    return fig

@app.callback(
    Output('sales_vs_returns', 'figure'),
    [Input('region_filter', 'value')]
)
def update_sales_vs_returns(selected_region):
    filtered_df = df[df['Region_ID'] == selected_region]
    return_data = filtered_df.groupby('Return_Flag')['Total_Sales'].sum().reset_index()
    fig = px.bar(return_data, x='Return_Flag', y='Total_Sales', title='Sales vs Returns')
    return fig

if __name__ == '__main__':
    app.run_server(debug=True)


In [12]:
import pandas as pd
import plotly.express as px
import dash
import dash_bootstrap_components as dbc
from dash import dcc, html
from dash.dependencies import Input, Output

# Load dataset
df = pd.read_csv("FreshMart Analytics - Orders.csv")

# Compute additional metrics
df['Profit_Margin'] = df['Profit'] / df['Total_Sales'] * 100

df['Order_Count'] = df['Order_ID'].nunique()  # Count unique orders
df['Average_Basket_Size'] = df['Quantity_Sold'] / df['Order_Count']
df['Sales_Growth'] = df['Total_Sales'].pct_change() * 100

# Categorize sales periods
def categorize_time(hour):
    if 6 <= hour < 12:
        return "Morning"
    elif 12 <= hour < 18:
        return "Afternoon"
    elif 18 <= hour < 24:
        return "Evening"
    else:
        return "Night"

df['Sales_Period'] = df['Hour'].apply(categorize_time)

# Initialize Dash app
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

# Define Layout
app.layout = dbc.Container([
    dbc.Row([
        # Left Column (Title, KPIs, Dropdown)
        dbc.Col([
            html.H1("FreshMart Sales Dashboard", className="text-center mb-4"),
            
            dbc.Card([
                dbc.CardBody([
                    html.H4("Total Sales", className="card-title"),
                    html.H2(f"${df['Total_Sales'].sum():,.2f}", className="card-text")
                ])
            ], className="mb-3"),
            
            dbc.Card([
                dbc.CardBody([
                    html.H4("Total Profit", className="card-title"),
                    html.H2(f"${df['Profit'].sum():,.2f}", className="card-text")
                ])
            ], className="mb-3"),
            
            dbc.Card([
                dbc.CardBody([
                    html.H4("Average Basket Size", className="card-title"),
                    html.H2(f"{df['Average_Basket_Size'].mean():.2f}", className="card-text")
                ])
            ], className="mb-3"),
            
            dbc.Card([
                dbc.CardBody([
                    html.H4("Sales Growth Rate", className="card-title"),
                    html.H2(f"{df['Sales_Growth'].mean():.2f}%", className="card-text")
                ])
            ], className="mb-3"),
            
            dcc.Dropdown(
                id='region_filter',
                options=[{'label': region, 'value': region} for region in df['Region_ID'].unique()],
                value=df['Region_ID'].unique()[0],
                clearable=False,
                className="mb-3"
            )
        ], width=4),
        
        # Vertical Line Separator
        dbc.Col(html.Div(style={'border-left': '2px solid #ddd', 'height': '100%'}), width=1),
        
        # Right Column (Graphs)
        dbc.Col([
            dcc.Graph(id='sales_trend'),
            dcc.Graph(id='top_products'),
            dcc.Graph(id='category_sales'),
            dcc.Graph(id='hourly_sales_heatmap'),
            dcc.Graph(id='profit_margin_trend'),
            dcc.Graph(id='sales_vs_returns')
        ], width=7)
    ])
], fluid=True)

# Callbacks to update graphs
@app.callback(
    Output('sales_trend', 'figure'),
    [Input('region_filter', 'value')]
)
def update_sales_trend(selected_region):
    filtered_df = df[df['Region_ID'] == selected_region]
    fig = px.line(filtered_df, x='Month', y='Total_Sales', color='Year', title=f'Sales Trend for Region {selected_region}')
    return fig

@app.callback(
    Output('top_products', 'figure'),
    [Input('region_filter', 'value')]
)
def update_top_products(selected_region):
    filtered_df = df[df['Region_ID'] == selected_region]
    top_products = filtered_df.groupby('Product_ID')['Total_Sales'].sum().nlargest(5).reset_index()
    fig = px.bar(top_products, x='Product_ID', y='Total_Sales', title=f'Top 5 Products in Region {selected_region}')
    return fig

@app.callback(
    Output('category_sales', 'figure'),
    [Input('region_filter', 'value')]
)
def update_category_sales(selected_region):
    filtered_df = df[df['Region_ID'] == selected_region]
    category_sales = filtered_df.groupby('Product_ID')['Total_Sales'].sum().reset_index()
    fig = px.pie(category_sales, names='Product_ID', values='Total_Sales', title='Sales by Product Category')
    return fig

@app.callback(
    Output('hourly_sales_heatmap', 'figure'),
    [Input('region_filter', 'value')]
)
def update_hourly_sales_heatmap(selected_region):
    filtered_df = df[df['Region_ID'] == selected_region]
    fig = px.histogram(filtered_df, x='Hour', y='Total_Sales', title='Hourly Sales Distribution', nbins=24)
    return fig

@app.callback(
    Output('profit_margin_trend', 'figure'),
    [Input('region_filter', 'value')]
)
def update_profit_margin_trend(selected_region):
    filtered_df = df[df['Region_ID'] == selected_region]
    fig = px.line(filtered_df, x='Month', y='Profit_Margin', color='Year', title='Profit Margin Trend')
    return fig

@app.callback(
    Output('sales_vs_returns', 'figure'),
    [Input('region_filter', 'value')]
)
def update_sales_vs_returns(selected_region):
    filtered_df = df[df['Region_ID'] == selected_region]
    return_data = filtered_df.groupby('Return_Flag')['Total_Sales'].sum().reset_index()
    fig = px.bar(return_data, x='Return_Flag', y='Total_Sales', title='Sales vs Returns')
    return fig

if __name__ == '__main__':
    app.run_server(debug=True)
